In [3]:
import pandas as pd
import numpy as np
from pathlib import Path

# =========================
# 0) Ayarlar
# =========================
DATA_DIR = Path("data/raw")          # kendi klasörüne göre değiştir
OUT_DIR  = Path("data/processed")
OUT_DIR.mkdir(parents=True, exist_ok=True)

GPS_FILE = "all_merged_data.csv"   # sende tam dosya adı neyse onu yaz

# Bu aralık: 19-25 Aralık (dahil)
DATE_START = pd.Timestamp("2025-12-19")
DATE_END   = pd.Timestamp("2025-12-25")

# GPS örnekleme aralığı ~10 sn denmiş; ama bazen gap olabilir.
# Mahalle süresi hesabında aşırı boşlukları şişirmemek için dt'yi bu üst limite kırpıyoruz.
DT_CAP_SECONDS = 300  # 5 dk

# Chunk okuma
CHUNK_SIZE = 200_000


# =========================
# 1) Yardımcı Fonksiyonlar
# =========================
def parse_hms_to_seconds(series: pd.Series) -> pd.Series:
    """
    'HH:MM:SS' veya benzeri formatları saniyeye çevirir.
    Bozuk/boş değerleri 0 yapar.
    """
    # pandas to_timedelta toleranslıdır; NaT -> NaN
    td = pd.to_timedelta(series, errors="coerce")
    return td.dt.total_seconds().fillna(0).astype(np.float64)

def parse_date_time(df: pd.DataFrame) -> pd.DataFrame:
    """
    Tarih + Saat -> datetime, ayrıca date alanı üretir.
    Tarih formatı veri setinde '19.12.2025' gibi görünüyor.
    """
    # Tarih ve Saat kolonlarını stringe çevirip birleştiriyoruz
    # dayfirst=True önemli (19.12.2025)
    dt = pd.to_datetime(
        df["Tarih"].astype(str).str.strip() + " " + df["Saat"].astype(str).str.strip(),
        errors="coerce",
        dayfirst=True
    )
    df["datetime"] = dt
    df["date"] = dt.dt.floor("D")
    return df

def in_range(df: pd.DataFrame) -> pd.DataFrame:
    """
    19-25 Aralık dahil filtre.
    """
    mask = (df["date"] >= DATE_START) & (df["date"] <= DATE_END)
    return df.loc[mask].copy()

def normalize_ids(df: pd.DataFrame) -> pd.DataFrame:
    """
    vehicle_id stringe çevir, trimle.
    """
    df["vehicle_id"] = df["vehicle_id"].astype(str).str.strip()
    return df


# =========================
# 2) Chunk Bazlı Baseline Toplama
# =========================
vehicle_daily_rows = []     # her chunk'tan günlük araç özetleri
# Mahalle süre hesabı için daha ince bilgi gerekiyor; onu ayrı toplayacağız
# (Araç-gün-mahalle sürelerini hesaplamak için segment yaklaşımı: dt)
veh_day_neigh_rows = []

# Mahalle blok (visit count) için araç-gün içerisinde sıralı geçiş gerekir.
# Chunklar araç-gün bazında parçalanabileceği için daha güvenli yaklaşım:
#   - önce "araç-gün" bazında mahalle sürelerini dt ile topluyoruz (yakın doğru)
#   - visit_count'u ikinci geçişte sadece küçük subset için hesaplayabiliriz
# Hackathon MVP'de visit_count opsiyonel; istersen sonra ekleriz.
#
# Şimdilik mahalle zamanı: dt yöntemi.

usecols = [
    "vehicle_id",
    "Enlem", "Boylam",
    "Duraklama Süresi", "Rölanti Süresi",
    "Durum",
    "Tarih", "Saat",
    "Hız(km/sa)",
    "Mesafe(km)",
    "Mesafe Sayacı(km)",
    "Adres",
    "Mahalle",
    "Kaynak"
]

reader = pd.read_csv(
    GPS_FILE,
    sep=None,
    engine="python",
    on_bad_lines="skip",
    chunksize=CHUNK_SIZE,
    usecols=lambda c: c in usecols
)


for i, chunk in enumerate(reader, start=1):
    # datetime / date
    chunk = parse_date_time(chunk)
    chunk = chunk.dropna(subset=["datetime"])  # parse edilemeyenleri at
    chunk = normalize_ids(chunk)

    # aralık filtre
    chunk = in_range(chunk)
    if chunk.empty:
        continue

    # süreleri saniyeye çevir
    chunk["idle_sec"] = parse_hms_to_seconds(chunk.get("Rölanti Süresi", pd.Series(dtype=str)))
    chunk["stop_sec"] = parse_hms_to_seconds(chunk.get("Duraklama Süresi", pd.Series(dtype=str)))

    # mesafe kolonları
    if "Mesafe(km)" in chunk.columns:
        chunk["dist_km"] = pd.to_numeric(chunk["Mesafe(km)"], errors="coerce").fillna(0.0)
    else:
        chunk["dist_km"] = 0.0

    if "Mesafe Sayacı(km)" in chunk.columns:
        chunk["odo_km"] = pd.to_numeric(chunk["Mesafe Sayacı(km)"], errors="coerce")
    else:
        chunk["odo_km"] = np.nan

    # ================
    # 2A) Günlük araç metrikleri (chunk içi toplama)
    # ================
    g = chunk.groupby(["date", "vehicle_id"], as_index=False).agg(
        total_dist_km=("dist_km", "sum"),
        total_idle_sec=("idle_sec", "sum"),
        total_stop_sec=("stop_sec", "sum"),
        first_ts=("datetime", "min"),
        last_ts=("datetime", "max"),
        # odometer varsa min-max (chunk içi) - sonra tüm chunklar birleşince tekrar min/max yapacağız
        odo_min=("odo_km", "min"),
        odo_max=("odo_km", "max"),
        records=("datetime", "size")
    )
    vehicle_daily_rows.append(g)

    # ================
    # 2B) Mahalle süresi hesabı için dt segmentleri
    # - Her satırın dt'si bir sonraki satıra kadar geçen süre.
    # - chunk bazında sıralama yapıyoruz; chunklar arası sınır hatası küçük olur, ama hackathon için kabul edilebilir.
    #   (İstersen daha sonra tek pass full sort ile %100 yaparız.)
    # ================
    chunk = chunk.sort_values(["vehicle_id", "date", "datetime"])
    # aynı vehicle_id ve date içindeki bir sonraki datetime
    chunk["next_dt"] = chunk.groupby(["vehicle_id", "date"])["datetime"].shift(-1)
    chunk["dt_sec"] = (chunk["next_dt"] - chunk["datetime"]).dt.total_seconds()
    chunk["dt_sec"] = chunk["dt_sec"].clip(lower=0, upper=DT_CAP_SECONDS).fillna(0.0)

    # mahalle boşsa ayırmak için
    chunk["Mahalle"] = chunk["Mahalle"].astype(str).str.strip()
    chunk.loc[chunk["Mahalle"].isin(["", "nan", "None"]), "Mahalle"] = np.nan

    # araç-gün-mahalle bazında süre toplama
    gn = chunk.dropna(subset=["Mahalle"]).groupby(["date", "vehicle_id", "Mahalle"], as_index=False).agg(
        time_in_neigh_sec=("dt_sec", "sum"),
        dist_in_neigh_km=("dist_km", "sum"),
        idle_in_neigh_sec=("idle_sec", "sum"),
        stop_in_neigh_sec=("stop_sec", "sum"),
        points=("dt_sec", "size")
    )
    veh_day_neigh_rows.append(gn)

    if i % 5 == 0:
        print(f"Chunk işlendi: {i}")


# =========================
# 3) Chunk Özetlerini Birleştir & Nihai Baseline Tablolarını Üret
# =========================
vehicle_daily = pd.concat(vehicle_daily_rows, ignore_index=True)

# Chunk içi topladığımız odo_min/max'ları tekrar araç-gün seviyesinde birleştiriyoruz
vehicle_daily_final = vehicle_daily.groupby(["date", "vehicle_id"], as_index=False).agg(
    total_dist_km=("total_dist_km", "sum"),
    total_idle_sec=("total_idle_sec", "sum"),
    total_stop_sec=("total_stop_sec", "sum"),
    first_ts=("first_ts", "min"),
    last_ts=("last_ts", "max"),
    odo_min=("odo_min", "min"),
    odo_max=("odo_max", "max"),
    records=("records", "sum")
)

# Odometer varsa, dist_km'yi daha güvenilir hale getirmek için "odo delta" üretebiliriz
vehicle_daily_final["odo_dist_km"] = vehicle_daily_final["odo_max"] - vehicle_daily_final["odo_min"]

# Eğer Mesafe(km) güvenilmezse (çok 0 vs), odometer delta'ya geçilebilir.
# Şimdilik ikisini de raporluyoruz.
vehicle_daily_final["work_duration_sec"] = (vehicle_daily_final["last_ts"] - vehicle_daily_final["first_ts"]).dt.total_seconds().fillna(0)

# saniyeleri daha okunur yapalım
vehicle_daily_final["total_idle_hr"] = vehicle_daily_final["total_idle_sec"] / 3600
vehicle_daily_final["total_stop_hr"] = vehicle_daily_final["total_stop_sec"] / 3600
vehicle_daily_final["work_duration_hr"] = vehicle_daily_final["work_duration_sec"] / 3600

# Kaydet
vehicle_daily_final.to_csv(OUT_DIR / "baseline_daily_vehicle_metrics.csv", index=False)


# ================
# 3B) Mahalle metrikleri (günlük mahalle toplamları)
# ================
veh_day_neigh = pd.concat(veh_day_neigh_rows, ignore_index=True)

# Mahalle-gün bazında tüm araçlardan birleştir
neigh_daily_final = veh_day_neigh.groupby(["date", "Mahalle"], as_index=False).agg(
    total_time_in_neigh_sec=("time_in_neigh_sec", "sum"),
    total_dist_in_neigh_km=("dist_in_neigh_km", "sum"),
    total_idle_in_neigh_sec=("idle_in_neigh_sec", "sum"),
    total_stop_in_neigh_sec=("stop_in_neigh_sec", "sum"),
    total_points=("points", "sum"),
    vehicles_served=("vehicle_id", "nunique")
)

neigh_daily_final["total_time_in_neigh_hr"] = neigh_daily_final["total_time_in_neigh_sec"] / 3600
neigh_daily_final["total_idle_in_neigh_hr"] = neigh_daily_final["total_idle_in_neigh_sec"] / 3600
neigh_daily_final["total_stop_in_neigh_hr"] = neigh_daily_final["total_stop_in_neigh_sec"] / 3600

neigh_daily_final.to_csv(OUT_DIR / "baseline_daily_neighborhood_metrics.csv", index=False)

# Opsiyonel: araç-gün-mahalle kırılımını da kaydedelim (ileride visit_count vs için yararlı)
veh_day_neigh.to_csv(OUT_DIR / "baseline_vehicle_neighborhood_time.csv", index=False)

print("Faz 1 tamam: baseline çıktıları yazıldı.")


Faz 1 tamam: baseline çıktıları yazıldı.


In [4]:
import pandas as pd
import numpy as np
from pathlib import Path

# =========================
# 0) Ayarlar (aynı klasör)
# =========================
BASE_DIR = Path(".")  # notebook hangi klasördeyse orası
GPS_FILE = BASE_DIR / "all_merged_data.csv"  # aynı klasörde

# çıktı dosyaları yine aynı klasöre yazılacak
OUT_VEHICLE = BASE_DIR / "baseline_daily_vehicle_metrics.csv"
OUT_NEIGH   = BASE_DIR / "baseline_daily_neighborhood_metrics.csv"
OUT_VEH_NEI = BASE_DIR / "baseline_vehicle_neighborhood_time.csv"

# Bu aralık: 19-25 Aralık (dahil)
DATE_START = pd.Timestamp("2025-12-19")
DATE_END   = pd.Timestamp("2025-12-25")

DT_CAP_SECONDS = 300   # dt üst limit (5 dk)
CHUNK_SIZE = 200_000

# =========================
# 1) Yardımcı Fonksiyonlar
# =========================
def parse_hms_to_seconds(series: pd.Series) -> pd.Series:
    td = pd.to_timedelta(series, errors="coerce")
    return td.dt.total_seconds().fillna(0).astype(np.float64)

def parse_date_time(df: pd.DataFrame) -> pd.DataFrame:
    dt = pd.to_datetime(
        df["Tarih"].astype(str).str.strip() + " " + df["Saat"].astype(str).str.strip(),
        errors="coerce",
        dayfirst=True
    )
    df["datetime"] = dt
    df["date"] = dt.dt.floor("D")
    return df

def in_range(df: pd.DataFrame) -> pd.DataFrame:
    mask = (df["date"] >= DATE_START) & (df["date"] <= DATE_END)
    return df.loc[mask].copy()

def normalize_ids(df: pd.DataFrame) -> pd.DataFrame:
    df["vehicle_id"] = df["vehicle_id"].astype(str).str.strip()
    return df

# =========================
# 2) Chunk Bazlı Baseline Toplama
# =========================
vehicle_daily_rows = []
veh_day_neigh_rows = []

usecols = [
    "vehicle_id",
    "Enlem", "Boylam",
    "Duraklama Süresi", "Rölanti Süresi",
    "Durum",
    "Tarih", "Saat",
    "Hız(km/sa)",
    "Mesafe(km)",
    "Mesafe Sayacı(km)",
    "Adres",
    "Mahalle",
    "Kaynak"
]

reader = pd.read_csv(
    GPS_FILE,
    sep=None,
    engine="python",
    on_bad_lines="skip",
    chunksize=CHUNK_SIZE,
    usecols=lambda c: c in usecols
)

for i, chunk in enumerate(reader, start=1):
    chunk = parse_date_time(chunk)
    chunk = chunk.dropna(subset=["datetime"])
    chunk = normalize_ids(chunk)

    chunk = in_range(chunk)
    if chunk.empty:
        continue

    chunk["idle_sec"] = parse_hms_to_seconds(chunk.get("Rölanti Süresi", pd.Series(dtype=str)))
    chunk["stop_sec"] = parse_hms_to_seconds(chunk.get("Duraklama Süresi", pd.Series(dtype=str)))

    chunk["dist_km"] = pd.to_numeric(chunk.get("Mesafe(km)", 0), errors="coerce").fillna(0.0)

    if "Mesafe Sayacı(km)" in chunk.columns:
        chunk["odo_km"] = pd.to_numeric(chunk["Mesafe Sayacı(km)"], errors="coerce")
    else:
        chunk["odo_km"] = np.nan

    # ---- 2A) Günlük araç metrikleri
    g = chunk.groupby(["date", "vehicle_id"], as_index=False).agg(
        total_dist_km=("dist_km", "sum"),
        total_idle_sec=("idle_sec", "sum"),
        total_stop_sec=("stop_sec", "sum"),
        first_ts=("datetime", "min"),
        last_ts=("datetime", "max"),
        odo_min=("odo_km", "min"),
        odo_max=("odo_km", "max"),
        records=("datetime", "size")
    )
    vehicle_daily_rows.append(g)

    # ---- 2B) Mahalle süresi (dt yaklaşımı)
    chunk = chunk.sort_values(["vehicle_id", "date", "datetime"])
    chunk["next_dt"] = chunk.groupby(["vehicle_id", "date"])["datetime"].shift(-1)
    chunk["dt_sec"] = (chunk["next_dt"] - chunk["datetime"]).dt.total_seconds()
    chunk["dt_sec"] = chunk["dt_sec"].clip(lower=0, upper=DT_CAP_SECONDS).fillna(0.0)

    chunk["Mahalle"] = chunk["Mahalle"].astype(str).str.strip()
    chunk.loc[chunk["Mahalle"].isin(["", "nan", "None"]), "Mahalle"] = np.nan

    gn = chunk.dropna(subset=["Mahalle"]).groupby(["date", "vehicle_id", "Mahalle"], as_index=False).agg(
        time_in_neigh_sec=("dt_sec", "sum"),
        dist_in_neigh_km=("dist_km", "sum"),
        idle_in_neigh_sec=("idle_sec", "sum"),
        stop_in_neigh_sec=("stop_sec", "sum"),
        points=("dt_sec", "size")
    )
    veh_day_neigh_rows.append(gn)

    if i % 5 == 0:
        print(f"Chunk işlendi: {i}")

# =========================
# 3) Nihai tablolar
# =========================
vehicle_daily = pd.concat(vehicle_daily_rows, ignore_index=True)

vehicle_daily_final = vehicle_daily.groupby(["date", "vehicle_id"], as_index=False).agg(
    total_dist_km=("total_dist_km", "sum"),
    total_idle_sec=("total_idle_sec", "sum"),
    total_stop_sec=("total_stop_sec", "sum"),
    first_ts=("first_ts", "min"),
    last_ts=("last_ts", "max"),
    odo_min=("odo_min", "min"),
    odo_max=("odo_max", "max"),
    records=("records", "sum")
)

vehicle_daily_final["odo_dist_km"] = vehicle_daily_final["odo_max"] - vehicle_daily_final["odo_min"]
vehicle_daily_final["work_duration_sec"] = (vehicle_daily_final["last_ts"] - vehicle_daily_final["first_ts"]).dt.total_seconds().fillna(0)

vehicle_daily_final["total_idle_hr"] = vehicle_daily_final["total_idle_sec"] / 3600
vehicle_daily_final["total_stop_hr"] = vehicle_daily_final["total_stop_sec"] / 3600
vehicle_daily_final["work_duration_hr"] = vehicle_daily_final["work_duration_sec"] / 3600

vehicle_daily_final.to_csv(OUT_VEHICLE, index=False)

veh_day_neigh = pd.concat(veh_day_neigh_rows, ignore_index=True)

neigh_daily_final = veh_day_neigh.groupby(["date", "Mahalle"], as_index=False).agg(
    total_time_in_neigh_sec=("time_in_neigh_sec", "sum"),
    total_dist_in_neigh_km=("dist_in_neigh_km", "sum"),
    total_idle_in_neigh_sec=("idle_in_neigh_sec", "sum"),
    total_stop_in_neigh_sec=("stop_in_neigh_sec", "sum"),
    total_points=("points", "sum"),
    vehicles_served=("vehicle_id", "nunique")
)

neigh_daily_final["total_time_in_neigh_hr"] = neigh_daily_final["total_time_in_neigh_sec"] / 3600
neigh_daily_final["total_idle_in_neigh_hr"] = neigh_daily_final["total_idle_in_neigh_sec"] / 3600
neigh_daily_final["total_stop_in_neigh_hr"] = neigh_daily_final["total_stop_in_neigh_sec"] / 3600

neigh_daily_final.to_csv(OUT_NEIGH, index=False)
veh_day_neigh.to_csv(OUT_VEH_NEI, index=False)

print("Faz 1 tamam.")
print("Yazılan dosyalar:")
print(" -", OUT_VEHICLE.resolve())
print(" -", OUT_NEIGH.resolve())
print(" -", OUT_VEH_NEI.resolve())


Faz 1 tamam.
Yazılan dosyalar:
 - C:\Users\sefas\OneDrive\Masaüstü\NB_hackathon_2025-main\baseline_daily_vehicle_metrics.csv
 - C:\Users\sefas\OneDrive\Masaüstü\NB_hackathon_2025-main\baseline_daily_neighborhood_metrics.csv
 - C:\Users\sefas\OneDrive\Masaüstü\NB_hackathon_2025-main\baseline_vehicle_neighborhood_time.csv
